<a href="https://colab.research.google.com/github/xaviguerrero93/ml-github/blob/branch1/Fund_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
User manual and recommendations

Source for tickers: https://finance.yahoo.com/

Source for fees: https://www.morningstar.com/

Source for inflation rates: https://www.inflation.eu/en/inflation-rates/cpi-inflation-2020.aspx

Features already tested and working:

1.   Adding/modifying actions/trackers/actives (available in yahoo finance)
2.   Modifying portfolio weights/inflation/fees
3.   Modifying evaluation day
4.   Modifying index wrt you want to compare your portfolio (ACWI or SP500)

For any suggestion, please contact xaviguerrerofx@gmail.com

**USER MANUAL**

Go to the portfolio, and modify it as you want. It is recommended to copy the portfolio and create a second one. Be sure to set active the portfolio you want.

You can modify:
- Tickers of your portfolio. It can be actions, trackers, futures... as long as it is available in yahoo finance.
- Weights of the tickers
- Inflation rates of each country, fees of each ticker
- Reference index to which compare your portfolio

Once your portfolio defined, it is recommended use the command 
- "Run all" if it is the first time you use the notebook
- "Restart and run all" otherwise.


# Library import
To launch every time you add/modify tickers

In [1]:
# To show two vars using the same cell without display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

# Libraries to import data
import matplotlib.pyplot as plt
import numpy as np
import os
import tarfile
import urllib.request as urllib
from urllib.error import HTTPError
import pandas as pd

yahoo_dir = "yahoo_data/"
if(os.path.isdir(yahoo_dir)):
  !rm -rf yahoo_data
!mkdir yahoo_data

# Portfolio 1
You can copy this whole section to define an entire new portfolio. Make sure that the boolean `portfolio_active` is set to `True`, but only for the portfolio to use.

In [2]:
portfolio_active = True

In [3]:
# Average inflations of each country (2020)
inflation = {}
inflation['USA'] = 1.24 #Inflation in percentage (CPI)
inflation['China'] = 2.44
inflation['Japan'] = 0.00
inflation['Europe'] = 0.25

In [4]:
################################## PORTFOLIO ###################################
#DATA TO BE COMPLETED BY THE USER

if(portfolio_active):
  ################################# TICKERS ####################################
  # Init
  tickers = [] 
  pf_weights = pd.DataFrame()
  fees = {}

  # Amundi SP500
  active = True                 #Set to true if it is part of your portfolio
  if(active):
    name = "PE500.PA"           #Name of the ticker in yahoo finance
    tickers.append(name)        
    fees[name] = 0.15           # Annualized fees in percentage
    pf_weights[name] = [0.49]   # Weight of this active in your portfolio
    inflation[name] = inflation['USA']

  # Lyxor Nasdaq
  active = True
  if(active):
    name = "PUST.PA" 
    tickers.append(name)         
    fees[name] = 0.30 
    pf_weights[name] = [0.14]    
    inflation[name] = inflation['USA']

  # Amundi Russell 2000
  active = False
  if(active):
    name = "RS2K.PA" 
    tickers.append(name)         
    fees[name] = 0.35
    pf_weights[name] = [0.05]    
    inflation[name] = inflation['USA']

  # Amundi Europe
  active = True
  if(active):
    name = "PCEU.PA" 
    tickers.append(name)
    fees[name] = 0.15 
    pf_weights[name] = [0.2]
    inflation[name] = inflation['Europe']

  # BNP Europe Small Caps
  active = False
  if(active):
    name = "EESM.PA" 
    tickers.append(name)
    fees[name] = 0.25
    pf_weights[name] = [0.0]
    inflation[name] = inflation['Europe']

  # Lyxor Asia pacific ex Japan
  active = True
  if(active):
    name = "PAEJ.PA" 
    tickers.append(name)
    fees[name] = 0.60
    pf_weights[name] = [0.1]    
    inflation[name] = inflation['China']

  # Amundi Japan topix
  active = True
  if(active):
    name = "PTPXE.PA" 
    tickers.append(name)
    fees[name] = 0.20 
    pf_weights[name] = [0.07]
    inflation[name] = inflation['Japan']

  ############################ END OF TICKERS ###################################
  pf_weights
  print("Sum of weights:",pf_weights.sum(axis=1).item())

  # Day you want to evaluate the portfolio, format dd/mm/yyyy. 
  # Evaluation will be done through one year, ending in eval_day
  # Eval day cannot be before 01/01/2018 or after 29/01/2021
  eval_day_str = "29/01/2021" 


  # Index to be compared with (comment the others, otherwise last row is used)
  #idx_str = "MSCI ACWI" # Worldwide index (including Asia)
  idx_str = "S&P 500"  # American economic index
  

,PE500.PA,PUST.PA,PCEU.PA,PAEJ.PA,PTPXE.PA
0,0.49,0.14,0.2,0.1,0.07


Sum of weights: 1.0


# Internal calculation
Make sure to launch it when modifying your portfolio
Steps:
1.   Preparing data of stickers and reference index
2.   Calculating index performance (return, volatility)
3.   Calculating portfolio performance (return, volatility, beta wrt index, correlation between actives)

In [5]:
############# SECTION 1 - PREPARING TRACKER DATA ###############################

if(pf_weights.sum(axis=1).item() == 1):
  pass
else:
  raise ValueError("The sum of weights is not equal to 1! Verify your weights")

# Set start and end date
from datetime import datetime, timedelta

eval_day = datetime.strptime(eval_day_str,'%d/%m/%Y') + timedelta(days=1)
begin_day = eval_day - timedelta(days=367)

url_evalday = str(int(datetime.timestamp(eval_day)))
url_beginday = str(int(datetime.timestamp(begin_day)))

td = "1d" #For daily comparison

# Download data of tickers (yahoo finance)
price = pd.DataFrame()
date_yahoo = pd.DataFrame()
try:
  for i in tickers:
    print("\n",i)
    
    url ="https://query1.finance.yahoo.com/v7/finance/download/"+i+"?period1="+url_beginday+"&period2="+url_evalday+"&interval="+td+"&events=history&includeAdjustedClose=true"

    urllib.urlretrieve(url, yahoo_dir+i+'.csv')
    price[i] = pd.read_csv(yahoo_dir+i+".csv")['Adj Close']
    date_yahoo[i] = pd.read_csv(yahoo_dir+i+".csv")['Date']
except HTTPError:
  raise ValueError("This ticker is not available!Verify availability in yahoo finance website")


# Convert tracker date to timestamp for comparison
first_col = pf_weights.columns.values[0]
date1 = []
for i in range(len(date_yahoo[first_col])):
  date1.append(datetime.timestamp(datetime.strptime(date_yahoo[first_col][i],'%Y-%m-%d')))
date_yahoo['timestamp'] = date1

# Drop rows where prices are not valid
idx = price.index[price[first_col].isnull()].tolist()
price.drop(index=idx,inplace=True)
price.reset_index(drop=True,inplace=True)
date_yahoo.drop(index=idx,inplace=True)
date_yahoo.reset_index(drop=True,inplace=True)

############# SECTION 2 - PREPARING INDEX DATA ################################

# Ensuring index name validity and loading/formatting data of the index

idx_ref = pd.DataFrame()
date1 = []
github_root = "https://raw.githubusercontent.com/xaviguerrero93/ml-github/branch1/Investing/"

if(idx_str == "MSCI ACWI"): 
  idx_file = pd.read_csv(github_root+"MSCI_ACWI_daily_4Y.csv")
  #Index1 - Create data
  idx_ref['original_date'] = idx_file['Date']
  idx_ref['price'] = idx_file['ACWI Standard (Large+Mid Cap)']
  date_format = '%b %d, %Y'

elif(idx_str == "S&P 500"):
  idx_file = pd.read_csv(github_root+"SP500_daily_5Y.csv")
  #Index2 - Create data
  idx_ref['original_date'] = idx_file['Date']
  idx_ref['price'] = idx_file[' Close/Last']
  date_format = '%m/%d/%Y'

else:
  raise NameError("Index name is not valid")

#Convert date to timestamp
for i in range(len(idx_ref['original_date'])):
  date1.append(datetime.timestamp(datetime.strptime(idx_ref['original_date'][i],date_format))) 
idx_ref['timestamp'] = date1

############# SECTION 3 - FILTERING VALID DATA ################################

# Compare timestamp and save dates available in tracker and index
idx2 = []
rowstodrop = []

for i in range(len(date_yahoo)):
  if date_yahoo['timestamp'][i] in idx_ref['timestamp'].values:
    idx2.append(idx_ref['original_date'].index[date_yahoo['timestamp'][i] == idx_ref['timestamp']].values.item())
  else:
    rowstodrop.append(i) #Raws not available in index are dropped

# Drop rows not available in the index
price.drop(index=rowstodrop,inplace=True)
price.reset_index(drop=True,inplace=True)
date_yahoo.drop(index=rowstodrop,inplace=True)
date_yahoo.reset_index(drop=True,inplace=True)

#Index treated prices
price_idx = pd.DataFrame()
price_idx[idx_str] = idx_ref['price'][idx2] 
price_idx.reset_index(drop=True,inplace=True)


 PE500.PA


('yahoo_data/PE500.PA.csv', <http.client.HTTPMessage at 0x7fef97b9a6a0>)


 PUST.PA


('yahoo_data/PUST.PA.csv', <http.client.HTTPMessage at 0x7fef97b9aa90>)


 PCEU.PA


('yahoo_data/PCEU.PA.csv', <http.client.HTTPMessage at 0x7fef97b9add8>)


 PAEJ.PA


('yahoo_data/PAEJ.PA.csv', <http.client.HTTPMessage at 0x7fef97b9a278>)


 PTPXE.PA


('yahoo_data/PTPXE.PA.csv', <http.client.HTTPMessage at 0x7fef97b9ad68>)

In [6]:
# Calculating percentage return for treated index
price_perc_idx = pd.DataFrame()
perc = [0]
for i in range(1,len(price_idx)):
  perc.append(100*(price_idx[idx_str][i] - price_idx[idx_str][i-1]) / price_idx[idx_str][i-1])
  
price_perc_idx[idx_str] = perc
perc.clear()

# Calculating percentage return for treated tickers
price_perc = pd.DataFrame()
for column in price:
  perc = [0]
  for i in range(1,len(price)):
    perc.append(100*(price[column][i] - price[column][i-1]) / price[column][i-1])
  price_perc[column] = perc
  perc.clear()

In [7]:
# CALCULATING INDEX PERFORMANCE
idx_perf = pd.DataFrame()
idx_perf['effective return'] = [100*(price_idx[idx_str].iloc[-1] - price_idx[idx_str][0]) / price_idx[idx_str][0]]
print(idx_str,"index effective return =",round(idx_perf['effective return'].item(),2),"%")

idx_perf['autobeta'] = price_idx[idx_str].cov(price_idx[idx_str],ddof=0)/price_idx.var(ddof=0).item()
print("Beta wrt himself (should be 1) =",round(idx_perf['autobeta'].item(),2))

# Portfolio volatility
idx_perf['volatility'] = price_perc_idx[idx_str].var(ddof=0)
print(idx_str,"index volatility =",round(idx_perf['volatility'].item(),2),"%")

S&P 500 index effective return = 13.47 %
Beta wrt himself (should be 1) = 1.0
S&P 500 index volatility = 4.75 %


In [8]:
# THIS SECTION CALCULATES EFFECTIVE RETURN OF THE PORTFOLIO AT THE EVAL DATE wrt ONE YEAR AGO

# Net returns of each tracker
net_return = {}
for column in price:
  net_return[column] = 100*(price[column].iloc[-1] - price[column][0]) / price[column][0]

# effective returns of each tracker
eff_return = {}
for column in price:
  eff_return[column] = net_return[column] - inflation[column] - fees[column] #LINE TO BE COMPLETED

# Portfolio effective return
pf = pd.DataFrame()
pf_efr = np.fromiter(eff_return.values(), dtype=float)
pf['Effective return'] = np.dot(pf_weights,pf_efr)
print("Portfolio effective return =",round(pf['Effective return'].item(),2),"%")

# THIS SECTION CALCULATES THE PORTFOLIO RISK

# Portfolio volatility
covmat = price_perc.cov(ddof=0)
pf['volatility'] = np.sqrt(np.dot(np.dot(pf_weights,covmat),np.transpose(pf_weights))).item()
print("portfolio volatility =",round(pf['volatility'].item(),2),"%")

# Portfolio beta wrt AMUNDI world
beta = []

for i in tickers:
  beta.append(price_perc_idx[idx_str].cov(price_perc[i],ddof=0)/price_perc_idx.var(ddof=0).item())

pf['beta'] = np.dot(pf_weights,beta).item()
print("portfolio beta (",idx_str,") =",round(pf['beta'].item(),2))


Portfolio effective return = 6.17 %
portfolio volatility = 1.57 %
portfolio beta ( S&P 500 ) = 0.49


# Results of the analysis
Run this section after internal calculations to evaluate your portfolio

In [9]:
# This section calculates portfolio correlation between MSCI world and each stock
for i in tickers:
  corr_val = price_perc_idx[idx_str].corr(price_perc[i]).item()
  print("Correlation between",idx_str,"and",i,":",round(corr_val,2))

# Portfolio correlation between stocks
print("\nCorrelation matrix for ETFs\n",price.corr())

Correlation between S&P 500 and PE500.PA : 0.57
Correlation between S&P 500 and PUST.PA : 0.57
Correlation between S&P 500 and PCEU.PA : 0.72
Correlation between S&P 500 and PAEJ.PA : 0.73
Correlation between S&P 500 and PTPXE.PA : 0.62

Correlation matrix for ETFs
           PE500.PA   PUST.PA   PCEU.PA   PAEJ.PA  PTPXE.PA
PE500.PA  1.000000  0.893600  0.909620  0.938683  0.929269
PUST.PA   0.893600  1.000000  0.673098  0.909323  0.819313
PCEU.PA   0.909620  0.673098  1.000000  0.844281  0.877171
PAEJ.PA   0.938683  0.909323  0.844281  1.000000  0.911729
PTPXE.PA  0.929269  0.819313  0.877171  0.911729  1.000000


In [10]:
# Final analysis

# Reference index
print(idx_str,":")
print("Effective return =",round(idx_perf['effective return'].item(),2),"%")
print("Volatility =",round(idx_perf['volatility'].item(),2),"%\n")

#Portfolio
print("PORTFOLIO:")
print("Weights\n",pf_weights,"\n")
print("Effective return =",round(pf['Effective return'].item(),2),"%")
print("Volatility =",round(pf['volatility'].item(),2),"%")
print("Beta* (",idx_str,") =",round(pf['beta'].item(),2))
#print("\nCorrelation matrix for ETFs\n",price.corr())
print("""\n\n*Beta corresponds to the sensibility of your portfolio wrt the 
index.If beta=0.7, for each 1% of increase/decrease of the index, 
your portfolio will increase/decrease by 0.7%""")

S&P 500 :
Effective return = 13.47 %
Volatility = 4.75 %

PORTFOLIO:
Weights
    PE500.PA  PUST.PA  PCEU.PA  PAEJ.PA  PTPXE.PA
0      0.49     0.14      0.2      0.1      0.07 

Effective return = 6.17 %
Volatility = 1.57 %
Beta* ( S&P 500 ) = 0.49


*Beta corresponds to the sensibility of your portfolio wrt the 
index.If beta=0.7, for each 1% of increase/decrease of the index, 
your portfolio will increase/decrease by 0.7%
